In [ ]:
import tweepy
import datetime
import pandas as pd
import time

In [ ]:
class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
    'consumer_key':        '',
    'consumer_secret':     '',
    'access_token_key':    '',
    'access_token_secret': ''
}
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dril", #BECAUSE WHO ELSE!
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                   #'retweet_text':    item.retweeted_status.full_text, ##throws error when not present
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data   

In [ ]:
miner=TweetMiner(result_limit=200)

In [ ]:
twitter_dict={}

In [ ]:
counter=0

for name in mp[mp['Locked']==0]['Screen Name']:
    twitter_dict[name]=[]
    twitter_dict[name].append(miner.mine_user_tweets(user=name, max_pages=17))
    counter = counter +1
    if counter%50==0:
        print(counter)
        print(name)
        time.sleep(900) #15 minute sleep time
        
    

In [ ]:
#check if list of users are valid
def remove_protected_accounts(users_df):
    working_df = users_df.copy()
    protected_ids = []
    
    counter = 0
    for index, row in users_df.iterrows():
        user = api.get_user(id=index)._json
        if user['protected'] == True:
            protected_ids.append(index)
        counter += 1
        if counter % 100 == 0:
            print(str(counter) + '/' + str(len(working_df)) + ' IDs checked')
            print('Protected IDs found:', len(protected_ids), '\n')
        # Sleeping for up to 0.2s helps not to exceed Tweepy's rate limit.
        time.sleep(random() / 5)
    
    working_df.drop(labels=protected_ids, inplace=True)
    working_df.reset_index(inplace=True)
            
    return working_df